In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import mglearn as mg
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.preprocessing import OneHotEncoder , StandardScaler , LabelEncoder , OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier , KNeighborsRegressor
from sklearn.model_selection import train_test_split , KFold , cross_val_score , GridSearchCV , TimeSeriesSplit , GroupKFold , learning_curve , validation_curve
from sklearn.linear_model import Lasso , LogisticRegression
from sklearn.metrics import mean_squared_error
import seaborn as sb
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
import scipy.stats as stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import ConfusionMatrixDisplay , accuracy_score , mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor , DecisionTreeClassifier
from statsmodels.stats.diagnostic import het_breuschpagan
import statsmodels.api as sm
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.base import clone
import xgboost as xgb
from xgboost import XGBClassifier , plot_importance
import shap


0: reading the data

In [ ]:
aisles = pd.read_csv(r"archive\aisles.csv")
department = pd.read_csv(r"archive\departments.csv")
pro_prior = pd.read_csv(r"archive\order_products__prior.csv")
pro_train = pd.read_csv(r"archive/order_products__train.csv")
orders = pd.read_csv(r"archive\orders.csv")
products = pd.read_csv(r"archive\products.csv")

1: Joins and memory optimization

In [ ]:
def reduce_memory(df):

    for col in df.columns:
        col_type = df[col].dtype

        if "int" in str(col_type):                 #السترينق حطيتها لانه برجع np.dtype ف لازم نحولها لنوعها المنطقي عشان نقدر نعمل مقارنه
            df[col] = df[col].astype("int32")

        elif "float" in str(col_type):
            df[col] = df[col].astype("float32")

        #بالبدايه حطيت بس else بعدين اكتشفت انه في انواع بيانات ثانيه مثل bool
        elif col_type == "object":
            df[col] = df[col].astype("category")

    return df

In [ ]:
dfs = [aisles , department , pro_prior , pro_train , orders , products]

for i in range(len(dfs)):
    dfs[i] = reduce_memory(dfs[i])

In [ ]:
all_products = pd.concat(
    [pro_prior, pro_train],
    axis=0,
    ignore_index=True
)


#   في m1
#   pro_prior بلشنا ب هذول لانه يحتوي تفاصيل المنتجات داخل الطلبات
#   orders هو اللي بحتوي على معلومات الطلب , المستخدم , الوقت
m1 = all_products.merge(orders, on="order_id", how="left")
m2 = m1.merge(products, on="product_id", how="left")
m3 = m2.merge(department, on="department_id", how="left")

Full_DataSet = m3.merge(aisles, on="aisle_id", how="left")



In [ ]:
print(Full_DataSet.info())

In [ ]:
#كنت بدي اصير اعمل قراءه للملف مره ثانيه عشان ما اضل اعمل دمج كل ما اشغل الكود , بس اكتشفت انه تحميله بقعد وقت اكثر
#Full_DataSet.to_csv('archive/full_instacart_data.csv', index=False)

2: EDA

In [ ]:
#cheking for null values
missing_values = Full_DataSet.isnull().sum()
missing_values = missing_values[missing_values > 0]
print( missing_values )
print("=="*40 )
missing_percent = (missing_values / len(Full_DataSet)) * 100
print(f"Percentage of missing values in columns\n {missing_percent}", )

missing_percent.plot(kind='bar', figsize=(8, 5), width=0.3, color='red', rot=0)
plt.ylabel("Missing Percentage")
plt.title("Missing Values Percentage per Feature")
plt.show()
# هاي مش قيم مفقوده بالغلط هاي بتدل انه الزبون اول مره بيطلب فمفيش عندو قيمه ل days_since_prior_order

In [ ]:
missing_percent = (missing_values / len(Full_DataSet)) * 100
print(f"Percentage of missing values in columns\n {missing_percent}", )

missing_percent.plot(kind='bar', figsize=(8, 5), width=0.3, color='red', rot=0)
plt.ylabel("Missing Percentage")
plt.title("Missing Values Percentage per Feature")
plt.show()
# هاي مش قيم مفقوده بالغلط هاي بتدل انه الزبون اول مره بيطلب فمفيش عندو قيمه ل days_since_prior_order

In [ ]:
missing_percent = (missing_values / len(Full_DataSet)) * 100
print(f"Percentage of missing values in columns\n {missing_percent}", )

missing_percent.plot(kind='bar', figsize=(8, 5), width=0.3, color='red', rot=0)
plt.ylabel("Missing Percentage")
plt.title("Missing Values Percentage per Feature")
plt.show()
# هاي مش قيم مفقوده بالغلط هاي بتدل انه الزبون اول مره بيطلب فمفيش عندو قيمه ل days_since_prior_order

Distribution plots for numeric features and target(s) (histogram, density).


In [ ]:
# لازم نختار الاعمده الرقميه الي الها معنى او بتفيدنا لو عملنا الها هستوغرام او دينستي بالاحرى مين مهم افهم الديستربويشن تبعو
Full_DataSet['order_hour_of_day'].plot(kind='hist', bins=24, figsize=(10, 6), title='Distribution of Orders by Hour of Day')

#Histogram (المدرج التكراري) رسمة أعمدة بتبين الكمية في كل فترة

plt.xlabel('Hour of Day (0 - 23)') # سمينا المحور عشان الدكتور يفهم

plt.ylabel('Frequency (Number of Orders)')
plt.show()

#بنلاحظ  فوق الوقت الي بكون فيه وقت الذروه للزباين متى بكون باليوم

In [ ]:
Full_DataSet['days_since_prior_order'].plot(kind='hist', bins=30, figsize=(7, 6), color='orange', title='Distribution of Days Since Prior Order')

plt.xlabel('Days Since Last Order')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#Weekly shoppers (peak at 7 days) and Monthly shoppers (peak at 30 days). The spike at 30 days also includes customers who haven't ordered for more than a month

In [ ]:
'''# 1. نأخذ عينة عشوائية (10% مثلاً) عشان الرام ما تنفجر
# هذا العمود يمثل "ساعات اليوم" من 0 لـ 23
sample_hours = Full_DataSet['order_hour_of_day'].sample(frac=0.1, random_state=42)

# 2. الرسم


plt.figure(figsize=(10, 6))

sample_hours.plot(kind='kde', color='green', title='Density Plot of Order Hour of Day')

plt.xlabel('Hour of Day (0-23)')
plt.xlim(0, 23) # عشان نحصر الرسمة في حدود اليوم
plt.show()

#الدينستي بعطينا هون تفاصيل اكثر او معبر اكثر بس بوخذ وقت اكثر للامانه '''


Categorical cardinality analysis (barplots / top-k frequencies).

In [ ]:
# first we need to do some cardinality checking for the catorgorical features
categorical_cols = Full_DataSet.select_dtypes(include=['category']).columns
categorical_cols



In [ ]:
Full_DataSet["product_name"].value_counts() # هون بنلاحظ انو في منتجات كتير متكرره يعني الكارديناليتي عاليه فهاض العامود مابفيدني اعملو فيجواليزشن

In [ ]:
# نعمل فحص للباقي
checking = ['department', 'aisle', 'product_name']

cardinality_counts = Full_DataSet[checking].nunique()
print("عدد الأنواع في كل عمود")
print(cardinality_counts)
# هون بنلاحظ الديبراتمنت فيه تنوع واطي فا بنقدر نعملو فيجواليز ونستفيد منه
# الممرات 134 يعتبر عاللي فامبنقدر نرسمو كلو رح نوخذ الاكثر تكرارا تمام نفس الحاله بنطبقها على اسماء المنتجات

In [ ]:
# what is eval_set refer to ?
Full_DataSet['eval_set'].value_counts()
# هون شفنا انو هاض العامود بحتوي على داتا بتمثل الطلبات القديمه فهاض الاشي مابفيدني اني اعملو فيجواليز

In [ ]:


#  بنرسمه كأعمدة Barplot
Full_DataSet['department'].value_counts().plot(kind='bar', figsize=(12,6) , color='blue')

plt.title('Total Orders per Department', fontsize=15)
plt.ylabel('Number of Orders')
plt.xlabel('Department Name')
plt.xticks(rotation=45, ha='right') # ميلنا الأسماء عشان نقرأها بوضوح
plt.show()

In [ ]:
top_aisles = Full_DataSet['aisle'].value_counts().head(25)
#  هون ممكن تسالني انت طيب كيف رتبتهم من الاكثر للاقل ؟ لان الميثود تاعت الفاليوز بالديفولت تاعها بترتبهم من الاكبر لاصغبر

top_aisles.plot(kind='bar',figsize=(12, 6), color='blue')


plt.title('Top 25 Best Selling aisles ', fontsize=14)
plt.ylabel('Number of Orders')
plt.xlabel('Aisle Name')
plt.xticks(rotation=45, ha='right') # ميلنا الكلام بزاوية 45 عشان ينقرأ بوضوح
plt.show()

In [ ]:
products_for_visuals = Full_DataSet['product_name'].value_counts().head(20) # هون اخذنا اكثر 20 بس عشان يصير مقروء بشكل احسن

products_for_visuals.plot(kind='bar', figsize=(12, 6), color='blue')
plt.title('Top 25 Best Selling Products ', fontsize=14)
plt.ylabel('Number of Orders')
plt.xlabel('Product Name')
plt.xticks(rotation=45, ha='right')
plt.show()

• Correlation matrix, heatmap and pairwise scatter plots for selected numeric features.

In [ ]:
#  هاي الخطوة بدنا نعرف شو العوامل اللي بتخلي الزبون يعمل اعادة طلب او reorder
# في هاي الحاله لازم نختار الاعمده الرقميه الي الها سلوك او بمعنى اصح الها معنى

In [ ]:
selected_features = [
    'order_number',
    'order_dow',
    'order_hour_of_day',
    'days_since_prior_order',
    'add_to_cart_order',
    'reordered' # هاض العامود الي بدنا نعرف شو العوامل الي بتأثر عليه مثل ماقلنا فوق
]

# 2. حساب المصفوفة (Correlation Matrix)
# الدالة .corr() هي العقل المدبر اللي بيحسب العلاقات
corr_matrix = Full_DataSet[selected_features].corr()


# annot=True: عشان يكتب الرقم جوا المربع
# cmap='coolwarm': ألوان (أحمر للحار/الموجب، أزرق للبارد/السالب)
# fmt='.2f': منزلتين عشريتين بس
plt.figure(figsize=(10, 7))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, square=False)

plt.title('Correlation Heatmap of Numeric Features', fontsize=15)
plt.show()

In [ ]:
# 1. نأخذ عينة صغيرة جداً (1000 سطر) عشان الرسم يكون خفيف وواضح
# Scatter Plot بيموت لو الداتا كبيرة
scatter_sample = Full_DataSet.sample(n=1000, random_state=42)

# 2. تحديد الأعمدة اللي بدنا نشوف علاقتها ببعض
# ركزنا على أهم 3 أعمدة عشان ما نضيع وقت
cols_to_plot = ['add_to_cart_order', 'days_since_prior_order', 'reordered']

# 3. رسم الـ Pairplot
# hue='reordered': عشان يلون النقاط (برتقالي للمكرر، أزرق للجديد)
sns.pairplot(scatter_sample[cols_to_plot], hue='reordered', palette='husl', height=3)

plt.suptitle('Pairwise Scatter Plots ', y=1.02)
plt.show()


In [ ]:
#طيب اللون الاخضر هون بمثل المنتجات المعاد شرائها بنلاحظ بالرسمه انو دايما بالبدايه ببدا الزبون يجيب اغراضو الي  متعود عليها
#بعدين بعد ما يجيبهم بجيب او بجرب اغراض جديده  مثل مابثمل اللون الزهري الي بمثل الاشياء الجديده
#الرسمة الشمال تحت النقط بتوريك إن المنتجات المكررة الخضراء دايما محجوز الها المقاعد الأولى بالسلة  بغض النظر عن كم يوم مر.

# الرسمة اليمين جبال بتأكد إن الناس بتتسوق بنظام أسبوعي أو شهري والمنتجات الجديدة والقديمة بتمشي على نفس هذا النظام.

• Time-of-day, day-of-week, and monthly seasonality plots

In [ ]:
hourrr=Full_DataSet['order_hour_of_day'].value_counts()
hourrr

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(16, 7))


# هون لازم قبل مانرسم نجهز الداتا لانها عباره عن مليون سطر فا لازم نرتبها اول حسب التكرار بعدين بنرتبها حسب الاندكس تصاعدني
hour_counts = Full_DataSet['order_hour_of_day'].value_counts().sort_index()
#لو تركناها هيك لاحظت رح تطلع معنا الرسمه الساعات من صفر ل 23 فا حيكون شوي مش مقروء الوضع فا افترحت لو بدنا نوري هالرسمه لاي حد لو نزبط الاندكس
#ونخليه بنظام am و pm
#بكون احسن ليش فقط عشان نخليه مقروء واريح للعين اكثر
labels_of_hours = [
    "12 AM", "1 AM", "2 AM", "3 AM", "4 AM", "5 AM",
    "6 AM", "7 AM", "8 AM", "9 AM", "10 AM", "11 AM",
    "12 PM", "1 PM", "2 PM", "3 PM", "4 PM", "5 PM",
    "6 PM", "7 PM", "8 PM", "9 PM", "10 PM", "11 PM"
]
# x=labels_of_hours (الساعات)
# y=hour_counts.values (عدد الطلبات)
#هون اخترنا بار بلوت عشان سريع وبرضو برسملنا ال 24 عمود بسرعه عاليه
sns.barplot(x=labels_of_hours, y=hour_counts.values, color='orange',)

plt.title('Time of day Orders', fontsize=15)
plt.xlabel('Hours: 12 AM - 11 PM  ', fontsize=12,color='blue')
plt.ylabel('Count', fontsize=12)
plt.show()

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 6))

# رح يطلع معنا 7 أرقام من 0 ـ 6
day_counts = Full_DataSet['order_dow'].value_counts().sort_index()

#بنرتبهم كمان مره زي ماعملنا فوق
 # الويك إند عند الاجانب ببدا من السبت فهو رح يكون رقم صفر
days_labels = ["Saturday",  "Sunday", "Monday",  "Tuesday", "Wednesday", "Thursday","Friday "]



sns.barplot(x=days_labels, y=day_counts.values, color="orange")

plt.title('Orders Day of Week ', fontsize=15)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Day', fontsize=12)

plt.show()
#بنلاحظ انو بالويكند اعلى طلبات وهاض المنطقي لانو الناس بتكون معطله

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 6))

#  هون رح نستخدم هاض الكولوم لانو انسب اشي للمنثلي لانو مافي عنا بالداتا سيت كولم عن الاشهر
days_of_month_counts = Full_DataSet['days_since_prior_order'].value_counts().sort_index()
#رح نستخدم اللاين بلوت عشان نوضح التغيرات بشكل افضل
sns.lineplot(x=days_of_month_counts.index, y=days_of_month_counts.values, marker='o', color='red', linewidth=2)

# 3. تحسين المحاور
plt.title('Monthly Seasonality', fontsize=14)
plt.xlabel('Days Since Last Order', fontsize=12)
plt.ylabel('Number of Orders', fontsize=12)
plt.show()
#بنلاحظ في قمه عند ال 7 و 30 يوم هاض يعني انو الزبون كل اسبوع غالبا بيجي يشتري وفي زباين بتيجي كل شهر او اكثر من شهر هاض كلو بنحط عند ال 30

3: cleaning


In [ ]:
del Full_DataSet["aisle"]
del Full_DataSet["department"]

print(Full_DataSet.isnull().sum())
print("---------------------------------------")

#كان هدفي اشوف ال نان من هون بس ما زبطت ف شفتها من الملف نفسه
Full_DataSet.head(10)
#اللي بين معي انه ال نان بكون موجود لكل اول اوردير بطلبه المستخدم\الزبون

In [ ]:
def check_outliers(Full_DataSet):

    out = ["order_id" , "product_id" , "user_id" , "aisle_id" , "department_id" , "eval_set"]
    Outliers_DF = []
    numeric_cols_all = Full_DataSet.select_dtypes(include=[np.number]).columns.tolist()


    for i in numeric_cols_all:
        if i not in out:
            Outliers_DF.append(i)

    col_length = len(Outliers_DF)
    row = (col_length // 3) + 1
    plt.figure(figsize=(20 , 5 * row))

    #لاني بحتاج index + value خلال التكرار لرسم subplots.
    #عشان هيك استخدمت enumerate
    for i , col in enumerate(Outliers_DF):
        plt.subplot(row , 3 , i + 1)

        plot_data = Full_DataSet[col].dropna().sample(n = min(100000 , len(Full_DataSet)))

        sb.boxplot(x = plot_data , color="lightblue")
        plt.title(col , fontsize = 12)
        plt.xlabel(" ")

    plt.tight_layout()
    plt.show()

In [ ]:
#print(check_outliers(Full_DataSet))

#من خلال الرسم بين معي انه الحد الفاصل بين اكبر قيمه والاوتلايرز هي 30
Full_DataSet["add_to_cart_order"] =  np.where(Full_DataSet["add_to_cart_order"] > 30 , 30 , Full_DataSet["add_to_cart_order"])
#ونفس المبدأ بنطبق هون
Full_DataSet["order_number"] =  np.where(Full_DataSet["order_number"] > 50 , 50 , Full_DataSet["order_number"])

print(Full_DataSet["add_to_cart_order"].describe())
print()
print(Full_DataSet["order_number"].describe())



In [ ]:
Full_DataSet.dtypes

In [ ]:
imputer = SimpleImputer(strategy = "constant" , fill_value = 0)
#بعد ما تفرجت عالداتا من الاكسل , اكتشفت انه النان موجوده بس عند اول طلب للمستخدم , يعني ما عنده طلب مسبق
#الموديل لما يشوف الصفر رح يقلك هاض المستخدم جديد , وعشان هيك ما عنده طلبات مسبقه
Full_DataSet["days_since_prior_order"] = imputer.fit_transform(Full_DataSet["days_since_prior_order"].values.reshape(-1,1))

DF = pd.DataFrame(Full_DataSet , columns = Full_DataSet.columns)

#DF.to_csv('archive/full_instacart_data.csv', index=False)
print(DF.isnull().sum())



4: feature engineering (mandatory list)

In [ ]:
#total orders per user يعني كل مستخدم كم مره فات المحل واشترى يعني كم طلب عملو بشكل كامل مش كم منتج اشتراه بحياتو
# طيب هون القروب باي بيجمعلي كل الداتا تبع كل يوزر ايدي لحال
#بعدين بقله لكل يوزر بدي كولوم الاوردر نمبر تمام هسا الاورودر نمبر لكل زبون بمثل كل زبون كم طلب عملو لحد الان فا لو اخذتلو ال ماكس
#لو اخذت الماكس رح تعطيني رقم اخر فاتوره والي بمثل عدد الطلبات الكلي لكل زبون طيب ممكن تسالني كان بمكاني اختار كاونت مش ماكس صح كلامك لكن
#الداتا الي عندي او كولوم الاوردر نمبر يعني بحتوي على ارقام الطلبات مش عدد الطلبات فلو اخذت كاونت رح يطلعلي عدد المنتجات مش عدد الطلباتؤ
#استخدمنا الريست اندكس عشان يرجعلي الداتا فريم مش سيرييز لانو قروب باي بيرجع سيرييز افتراضيا وبتخرب الداتا بصير اليوزر هو كولوم الاندكس
user_total_orders = DF.groupby('user_id')['order_number'].max().reset_index()
# 3. بنسمي العمود اسم واضح عشان ما نتخربط بعدين
user_total_orders.columns = ['user_id', 'user_total_orders']

print(user_total_orders.head())

In [ ]:
#  average basket size
#هذه الميزة بتحدد القدرة الشرائية Purchasing Power
#وبتحدد نمط التسوق عند الزبون يعني بعرف الزبون الي عادته يشتري مثلا 50 غرض هاض زبون بمتوسط سلة كبيرة فا بهمني انو المودل ممكن
#بهمني انو المودل ممكن يشوف هالشي ويستفيد منه بحيث لو كان شاري 3 اغراض بس يضل يقترح عليه لانو هالزبون من عادتو يشتري كثير

#============================================================================================================

#طيب السواال كيف بنحسبها ؟ بنحسبها عن طريق انو بنجيب لكل زبون كم المنتجات الي اشتراها بشكل كلي
#بعدين بنجيب كم طلب عملو بشكل كلي
#بعدين بنقسم المجموع على العدد
#============================================================================================================
#هون اولا عشان نجيب مجموع المنتجات الي اشتراها كل زبون بنجيب كولوم البروودكت ايدي وبنعمللو كاونت هيك بنعد كل المنتجات الي اشتراها
#بعدين بنجيب كولوم الاوردر نمبر وبناخد الماكس زي ما شرحنا فوق عشان نعرف كم طلب عملو بشكل كلي

#واستخدمنا ميثود الاقريقيت الي بتتيحلي اعمل اكشنز متعدده على كولمز مختلفه بنفس الوقت بدل ما اعمل قروب باي مرتين
basket_data = DF.groupby('user_id').agg({ 'product_id':'count', 'order_number': 'max'}).reset_index()

# مجرد تسميت الكولمز بشكل واضح بس
basket_data.columns = ['user_id', 'total_items_bought', 'total_orders_made']

#الحسبه
basket_data['avg_basket_size'] = basket_data['total_items_bought'] / basket_data['total_orders_made']

print(basket_data.head())

In [ ]:
#  User-Level Features full
#هون لازم نعمل سورت اول عشان اخر مطلوب اللاست لاخر طلب فا ممكن يكون اخر طلب الي بجيبو مش هو اخر طلب عملو
#بس الجهاز عندي بتحملش
# Full_DataSet.sort_values(['user_id', 'order_number'], inplace=True)
user_features = DF.groupby('user_id').agg({
      # 1. Total #Orders
    'order_number': 'max',
    # 2. هاض بساعدنا نعرف كم منتج اشتراها كل زبون بشكل كلي
     'product_id': 'count',
    # 3. Reorder Ratio
    'reordered': 'mean',

   #هون حسبنا اخر مطلوبين بخطوه وحده بدل ما نعمل قروب باي مرتين
    'days_since_prior_order': ['mean', 'last']
}).reset_index()


user_features.columns = ['user_id',  'user_total_orders',  'user_total_items', 'user_reorder_ratio','user_avg_days_between', 'user_days_since_last_order'  ]

# Basket Size
user_features['user_avg_basket_size'] = user_features['user_total_items'] / user_features['user_total_orders']

#هاض العامود مش ضروري بعد ما حسبنا الافريج مابنحتاجه
del user_features['user_total_items']

display(user_features.head())

Product-level features

In [ ]:
# --- Product-Level Features ---

# هالمره التجميع رح يكون حسب المنتج مش حسب اليوزر
product_features = DF.groupby('product_id').agg({

    #  Popularity
    #هون شو ممكن يفيدنا هون قصدو من البوبولاريتي انه نعرف كم مرة انباع هاد المنتج ممكن من خلاله نعرف  اكثر المنتجات بينباع او وين اقل منتج
    'user_id': 'count',
    # ليش user_id؟ وليش count؟
    # عشان نعد كم زبون اشترى هاد المنتج لانو كل سطر بيمثل  عنا عمليه شراء.
    #====================================================================================
    #  Reorder Rate
    #هون بنحسب نسبة اعادة الطلبات للمنتج
    'reordered': 'mean',
    #هاي واضحه من اسمها بدهاش اشي
    #====================================================================================
    # Average  Position
    #طيب ممكن تسالني ليش اخترنا هاض الكولوم بالذات رح لانه بمثل ترتيب المنتج داخل السلة فلو اخذنا المين رح يعطينا ترتيب المنتج داخل السلة زي ماهو طالب
    'add_to_cart_order': 'mean'

}).reset_index()

product_features.columns = [ 'product_id', 'product_total_purchases', 'product_reorder_rate', 'product_avg_cart_position' ]

display(product_features.head())

User×Product interaction features

In [ ]:
#UserxProduct Interaction Features
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

#دمجتها من هسا عشان اجرب اشغل user_days_since_last_order لانها مش جزء من الداتا الاساسيه ف ما اشتغل الكود
DF = DF.merge(user_features[['user_id' , 'user_days_since_last_order']] , on = 'user_id', how = 'left')

prior_df = DF[DF['eval_set'] == 'prior'].copy()
# uxp_features = DF.groupby(['user_id', 'product_id']).agg({

uxp_features = prior_df.groupby(['user_id', 'product_id']).agg({
# Total Purchases of Product by User
#هون عشان نجيبها كان ممكن نستخدم اي عمود بس بدنا واحد  بس المهم نعد كم مرة هاد الزبون اشترا هاد المنتج
#طب ليش اخترت هاض الكولوم بالذات اختصارا للوقت بس عشان المطلوب الثاني رح ارجع اطلبه تمام
#====================================================================================

# Reorder Probability of Product by User
#هون استخدمنا برضو بنفس العامود بس اخذنا المين عشان يعطينا النسبة الي طلبها الزبون من هاد المنتج
    'reordered': ['count', 'mean'],

    'user_days_since_last_order': 'max'    # هاي مش عارف اعملها علقت كيف ممكن نجيب ال   days since last purchase by     user_days_since_last_order  days_since_prior_order
}).reset_index()

# 2. تسمية الأعمدة بشكل واضح (عشان الدكتور يفهم كل عمود شو هو)
uxp_features.columns = [
    'user_id',
    'product_id',
    'uxp_total_bought',
    'uxp_reorder_ratio',
    'uxp_days_last_order_'
]

display(uxp_features.head(20))

Temporal features: hour/day/month/year, season, holiday flags (if available).

In [ ]:
# طيب هون المطلوب منا بالتيمبورال فيشترز انو نزبط الوقت مثلا نخليه صبح ومسا ومثلا الايام نقسمها ايام عاديه وايام عطله والشهر
#بس مبادايا الشهر والسنه والموسم مابنقدر لانو مامعنا معلومات عنها  في رح نكتفي بالساعات واليوم
#نبدا بالساعات هسا التوقيت  عنا من 0 ل 23 فممكن نقسمهم لثلاث فترات
def time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Midnight'
DF['time_of_day'] = DF['order_hour_of_day'].apply(time_of_day)
print(DF['time_of_day'].head(10))
print("=="*40)
#طيب قلنا للايام بنقسمها لايام عاديه وايام عطله
#وزي مابنعرف بالداتا الي عندي السبت والاحد همه العطله فرقمهم بكون 0 و 1
#هون اختصارا على حالنا لقدام خليت الكولوم الجديد عباره عن ارقام 0 و 1 بدل ما اخليها نصوص عشان اسهل التعامل معها بعدين
def day_type(day):
    if (day == 0) or (day == 1):
        return 1 # طبعا واحد بتعني انها شسمو ويكند
    else:
        return 0
DF['is_weekend'] = DF['order_dow'].apply(day_type)
print(DF['is_weekend'].head(30))
#حطيت 30 لانو خفت كلها صفار ههههه يسعد ربك


In [ ]:
# --- 5. Aggregations over Windows (Last 3 Orders) - بدون Lambda ---

# 1. تجهيز الداتا: بنحسب حجم السلة لكل طلب (جدول صغير وخفيف)
orders_summary = DF.groupby(['user_id', 'order_number']).size().reset_index(name='basket_size')

# 2. الترتيب (مهم جداً): عشان لما نقول "آخر 3" يكونوا عنجد آخر 3 زمنياً
orders_summary = orders_summary.sort_values(['user_id', 'order_number'])

# 3. تعريف الفنكشن العادي (بدل اللمدا)
# هذا الفنكشن بياخذ عمود أرقام، وبحسب المتوسط المتحرك لآخر 3 قيم
def calculate_last_3_avg(series):
    # window=3: يعني خذ 3 قيم
    # min_periods=1: يعني حتى لو الزبون عنده طلب واحد بس، احسبله المعدل (ما ترجع Null)
    return series.rolling(window=3, min_periods=1).mean()

# 4. تطبيق الفنكشن على كل زبون
# transform: بتمسك الفنكشن اللي كتبناه فوق، وبتطبقه على كل "مجموعة" (زبون)
orders_summary['rolling_avg_3_orders'] = orders_summary.groupby('user_id')['basket_size'].transform(calculate_last_3_avg)

# 5. النتيجة النهائية
# احنا بهمنا "آخر وضع" وصله الزبون، فبناخذ آخر سطر لكل زبون
user_window_features = orders_summary.groupby('user_id').last().reset_index()

# ترتيب وتنظيف الجدول النهائي
user_window_features = user_window_features[['user_id', 'rolling_avg_3_orders']]
user_window_features.columns = ['user_id', 'u_avg_basket_last_3']

display(user_window_features.head(20))

✅ تم حساب (Rolling Window) باستخدام فنكشن عادي!

At least one engineered non-linear feature : log transforms

In [ ]:
# افترض إنك حسبت user_features في الخطوة الأولى
# بدنا نحول عمود "عدد الطلبات" باستخدام اللوغاريتم
#هسا
print("=")
#هون طقعت ديسبلاي لانها اوضح بس من برنت
user_features['u_total_orders_log'] = np.log(user_features['user_total_orders'])

# حطيناهم جمب بعض عشان تشوف الفرق
print("user_total_orders trasform")
display(user_features[['user_total_orders', 'u_total_orders_log']].head(90))
#
#كمان فيتشر ثاني نعملو مش غلط
product_features['p_total_purchases_log'] = np.log(product_features['product_total_purchases'])

print("="*40)
print("product_ total_purchases transform")

display(product_features[['product_total_purchases', 'p_total_purchases_log']].head())

In [ ]:
#بدي اسوي نسخه للاحتياط , هسا انا صرت بمرحله حرجه شوي و اي خطأ ممكن يدمر الداتا كامله ف الاحتياط واجب
final_df = DF.copy()

final_df = final_df.merge(user_features , on = 'user_id' , how = 'left')
#لاني دمجتهم من قبل , عملت هالحركه عشان اتأكد ما يصير عندي اي تكرار
final_df = final_df.drop(columns=[col for col in user_features.columns if col in final_df.columns and col != 'user_id'])

final_df = final_df.merge(user_features , on = 'user_id' , how = 'left')
final_df = final_df.merge(product_features , on = 'product_id' , how = 'left')
final_df = final_df.merge(uxp_features , on = ['user_id' , 'product_id'] , how = 'left')
final_df = final_df.merge(user_window_features , on = 'user_id' , how = 'left')

In [ ]:
print(final_df.shape)
print(DF.shape)
print()
print(final_df.isnull().sum())
final_df.head()

In [ ]:
#بدي اختار الاعمده اللي لازم ازبط الميموري الهم
final_df.info()

In [ ]:
def reduce_memory_FE(df , col_name):

    for col in col_name:
        col_type = df[col].dtype

        if "int" in str(col_type):
            df[col] = df[col].astype("int32")

        elif "float" in str(col_type):
            df[col] = df[col].astype("float32")

        elif col_type == "object":
            df[col] = df[col].astype("category")

    return df

In [ ]:
col_name = ["uxp_reorder_ratio" , "u_avg_basket_last_3" , "uxp_total_bought" , "p_total_purchases_log" , "product_avg_cart_position" , "product_reorder_rate" , "product_total_purchases" , "u_total_orders_log" , "user_avg_basket_size" , "user_reorder_ratio" , "is_weekend" , "time_of_day"]

final_df = reduce_memory_FE(final_df, col_name)

In [ ]:
final_df.info()

In [ ]:
final_df = final_df.drop(columns=['user_days_since_last_order_x' ,
                                  'user_days_since_last_order_y'] ,
    errors='ignore'
)

#ما عرفت اقلل الذاكره اكثر من هيك 😢
final_df.info()

In [ ]:
#final_df.to_csv('archive/new_instacart_data.csv', index=False)

5: Dimensionality & collinearity

In [ ]:
id_cols = ['order_id' , 'user_id' , 'product_id' , 'aisle_id' , 'department_id']
low_cols = ["department_id" , "order_dow" , "time_of_day"]
high_cols = ["user_id" , "product_id" , "aisle_id"]

final_df[high_cols] = final_df[high_cols].astype(str)
#هالحركه سويتها بعد ما متت وانا بحلل الكود بعد ما طلعلي التنبيه هاض
#Warning: No categorical columns found. Calling 'transform' will only return input data.
#لما راجعت الانكوديرز تذكرت انه التارقيت ما بشتغل غير مع نصوص والاعمده اللي انا معطيه اياهم رقميات

target_col = "reordered"
Frequency_col = "product_name"

#عدد الاعمده كبير جدا فقلت بعمل لوب + استثناءات عشان اريح راسي
num_cols = (final_df.drop(columns=[target_col]).select_dtypes(include=["int32" , "float32"]).columns.tolist())
num_cols = [c for c in num_cols if c not in id_cols]

In [ ]:
#بدي استخدم VIF , هاض عباره عن فنكشن رياضي ببين قديش في ارتباط وتكرار بين الاعمده نفسهم
#الهدف منه اني اشوف شو في اعمده فيهم تشابه كبير وبقدمو نفس المعلومه تقريبا عشان احذف واحد منهم

SAMPLE_SIZE = 50000
V = final_df[num_cols].sample(n=SAMPLE_SIZE , random_state = 42)

vif = pd.DataFrame()
vif["feature"] = V[num_cols].columns
vif["VIF"] = [variance_inflation_factor(V.values , i) for i in range(V.shape[1])]

#تحت 5 ممتاز
#بين ال 6 وال 10 مقبول
#اكثر من هيك بدك تشوف شو و وين في ترابط غير مهم وتبلش تحذف
#ال inf حذف مباشره

vif.sort_values("VIF" , ascending = True)

In [ ]:
plt.figure(figsize = (8 , 5))
plt.scatter(vif["VIF"] , vif["feature"])
plt.axvline(10 , color = 'red' , linestyle = '--')
plt.xlabel("VIF")
plt.title("VIF Scatter Plot")
plt.show()


In [ ]:
#حذفت القيم اللانهائيه , والقيم اللي فيها النسبه عاليه , لانهم بدلو على تكرار وتشابه المعلومات , يعني لو خليتهم كلهم زي كأني مكرر نفس العامود ما فرقت
drop_cols = [
    "user_days_since_last_order" ,
    "uxp_days_last_order_" ,
    #u_total_orders_log كنت بدي اخليه بما انه تعبنا عليه بمرحلة الهندسه بس النسبه فيه كانت كثير عاليه :(
    "user_total_orders" , 
    "user_reorder_ratio" ,
    "product_reorder_rate" ,
    "product_avg_cart_position"
    ]


    

final_df = final_df.drop(columns = drop_cols , errors="ignore")

#صار عندي ايرور بالبريبروسيسر لانه الداتا صار فيها عدم تطابق بعد الدروب ف بدي ارجع انسخ الاعمده كمان مره
num_cols = (final_df.drop(columns=[target_col]).select_dtypes(include=["int32" , "float32"]).columns.tolist())
num_cols = [c for c in num_cols if c not in id_cols]

6+7: preprocessing and scaling

In [ ]:
def best_params_for_TE(DF , high_cols):

    #المشكله اللي صارت انه لما اقسم الداتا , لسا ما شغل البريبروسيسر عليها ف لسا مش كل الفيتشرز تحولو لقيم رقميه
    del DF["product_name"]

    unique_users = DF['user_id'].unique()
    selected_users = np.random.choice(unique_users , size = 3000 , replace=False)
    #رفعت عدد العينات ل 5000 والكود طول لتنه اشتغل ف عشان هيك قللتهم ك حل وسط وهون طلع معي افضل نتيجع بعد عدة تكرارات

    df_check = DF[DF['user_id'].isin(selected_users)].copy()

    df_check[high_cols] = df_check[high_cols].astype(str)

    xc = df_check.drop("reordered", axis=1)
    yc = df_check['reordered']

    xc_train , xc_test , yc_train , yc_test = train_test_split(xc , yc , test_size = 0.2 , random_state = 42)

    pipeline = make_pipeline(
        ce.TargetEncoder(cols=high_cols) ,
        RandomForestClassifier(
            n_estimators = 100,  # عدد الأشجار
            max_depth = 10 ,      # عمق الشجرة (عشان ما يوخذ وقت طويل)
            random_state = 42 ,
            n_jobs = -1
        )
    )

    param_grid = {
        'targetencoder__smoothing': [1, 10, 50] ,
        'targetencoder__min_samples_leaf': [1, 20]
    }

    grid_search = GridSearchCV(
        estimator = pipeline ,
        param_grid = param_grid ,
        cv = 3 ,
        scoring = 'roc_auc' ,
        verbose = 1
    )

    print("\n")
    grid_search.fit(xc_train, yc_train)

    print("\n" + "=" * 30)
    print(f"Best ROC_AUC: {grid_search.best_score_:.4f}")
    print("Best Parameters:")
    print(grid_search.best_params_)

    #تجربه فاشله لتحديد افضل المعاملات
    #السبب انه عدد العينات كبير نسبيا ومدامني بجرب كل رقم بلوب لحال ف رح يوخذ مني وقت كبيييييير جدااااا
    '''
    smoothing_op = [1, 2, 10, 20, 50, 100]
    leaf_op = [1, 5, 10, 20, 50]
    Kfold_op = [3, 5, 10]

    results = []

    print("-" * 50)

    for k in Kfold_op:
        current_kf = KFold(n_splits=k, shuffle=True, random_state=42)

        for sm in smoothing_op:
            for leaf in leaf_op:

                encoder = ce.TargetEncoder(
                    cols=high_cols,
                    min_samples_leaf=leaf,
                    smoothing=sm
                )

                model = LogisticRegression(solver='liblinear')

                pipeline = make_pipeline(encoder, model)

                try:
                    scores = cross_val_score(pipeline, xc, yc, cv=current_kf, scoring="roc_auc")
                    mean_auc = scores.mean()
                    std_auc = scores.std()

                    results.append({
                        'n_splits': k,
                        'smoothing': sm,
                        'min_samples_leaf': leaf,
                        'auc_mean': mean_auc,
                        'auc_std': std_auc
                    })

                    print(f"K={k}, Smooth={sm}, Leaf={leaf} -> AUC: {mean_auc:.4f}")

                except Exception as e:
                    print(f"Error at K={k}, Smooth={sm}, Leaf={leaf}: {e}")

    results_df = pd.DataFrame(results)

    sorted_results = results_df.sort_values(by='auc_mean', ascending=False)

    print("\n")
    print("TOP 3 PARAMETER COMBINATIONS:")
    print(sorted_results.head(3))
    print()

    if not sorted_results.empty:
        best_params = sorted_results.iloc[0]
        print(f"\nthe best:\nSmoothing: {best_params['smoothing']}\nMin Samples Leaf: {best_params['min_samples_leaf']}\nK-Fold Splits: {int(best_params['n_splits'])}")
    '''

In [ ]:
def which_scaler(num_cols):

    SAMPLE_SIZE = 500000
    df_sampled = DF.sample(n=SAMPLE_SIZE, random_state=42).copy()

    fig, axes = plt.subplots(len(num_cols), 2, figsize=(14, 4 * len(num_cols)))
    plt.subplots_adjust(hspace=0.4, wspace=0.3)

    for i, col in enumerate(num_cols):

        sns.histplot(
            df_sampled[col],
            kde=True,
            ax=axes[i, 0],
            color='skyblue',
            edgecolor='black',
            line_kws={'linewidth': 3}
        )
        axes[i, 0].set_title(f'Distribution of {col}', fontsize=12)
        axes[i, 0].set_xlabel(col, fontsize=10)
        axes[i, 0].grid(axis='y', linestyle='--', alpha=0.7)

        stats.probplot(
            df_sampled[col].dropna(),
            dist="norm",
            plot=axes[i, 1]
        )
        axes[i, 1].set_title(f'Q-Q Plot of {col}', fontsize=12)
        axes[i, 1].set_xlabel('Theoretical Quantiles (Normal)', fontsize=10)
        axes[i, 1].set_ylabel('Sample Quantiles', fontsize=10)

    plt.savefig('numerical_features_distribution_analysis.png', bbox_inches='tight')
    print("تم حفظ تحليل التوزيعات في 'numerical_features_distribution_analysis.png'")
    plt.show()

In [ ]:
'''
num_cols = ["order_hour_of_day", "days_since_prior_order", "add_to_cart_order", "order_number"]
low_cols = ["department_id", "order_dow"]
high_cols = ["user_id", "product_id", "aisle_id"]
target_col = "reordered"
Frequency_col = "product_name"

#الرسم ببينلك انه التوزيع مبعثر وغير طبيعي وبحتوي على اوتلايرز كثيييييررر ,  عشان هيك كان افضل خيار استخدام
#SD لانه افضل بالتعامل مع الاوتلايرز وما بتأثر فيهم بشكل واضح وسلبي

#which_scaler(num_cols)
'''

In [ ]:
print("Numeric Columns:" , len(num_cols))
print(num_cols)

#بعد اكثر من تجربه افضل ناتج طلعلي كن زي اللي حطيتهم بالبريبروسيس
#best_params_for_TE(final_df , high_cols)

8: Imbalanced data handling (classification)

In [ ]:
final_df.info()

اللي قاعد بصير هسا اني نقلت الانكوديرز لبعد مرحلة الهندسه , ليش ؟
لانه بكل بساطه احترت كيف فعليا المفروض نسوي انكودينق للاعمده الجديده اللي عملناهم بعد الهندسه وكان هاض الحل الوحيد المنطقي ++ ما بزبط اسوي
VIF وانا عامل انكودينق

In [ ]:
"""#الموضوع هاض عباره عن وجود عدم توازين في تصنيف البيانات , مثلا الكلاس الاول نسبته اعلى من الثاني , ف هيك الموديل بصير يعتمد عالاول ويهمل الثاني
#ف هيك بصير الموديل فاشل باكتشاف الانماط الجديد

#تجهيز بيانات التصنيف#

#جبت عينات من الداتا لانه اللابتوب شلف عندي لما اشتغلت عالداتا كامله , حاولت اكبر نسبة العينات قد ما بقدر
SAMPLE_SIZE = 1000000
final_sample = final_df.sample(n = SAMPLE_SIZE , random_state = 42)

x_c = final_sample.drop("reordered", axis=1)
y_c = final_sample["reordered"]

xc_train , xc_test , yc_train , yc_test = train_test_split(x_c , y_c , test_size = 0.3 , stratify = y_c , random_state = 42)

#طلعلي هيك SettingWithCopyWarning
xc_train = xc_train.copy()
xc_test  = xc_test.copy()

#هالحركه سويتها بعد ما متت وانا بحلل الكود بعد ما طلعلي التنبيه هاض
#Warning: No categorical columns found. Calling 'transform' will only return input data.
#لما راجعت الانكوديرز تذكرت انه التارقيت ما بشتغل غير مع نصوص والاعمده اللي انا معطيه اياهم رقميات

train_cls = preprocessor.fit_transform(xc_train , yc_train)
test_cls = preprocessor.transform(xc_test)

''''''''''''''''''''''''''''''''''''''''''''''''
#طيب هون بالتايم اوير سبلتنق المفروض نقسم الداتا حسب التوقيت طيب ليش ؟
#الداتا الجديده بقصد فيها الي صارت اخر شي او الطلبات الجديده او يعني الي اخر اشي عمله الزبون
#اولا عشان التقسيم العشوائي بالداتا تبعتنا بعمللنا مشكله شو المشكله هي انو المودل لما يجي يعمل فيت للداتا رح يعمل فيت ويتعلم على داتا جدي
#بعد مايعمل فيت على داتا ممكن تكون جديده ممكن يكون التيست عنا داتا قديمه فهيك المودل رح يكون متعلم من داتا الجديده وهيك بغش حالو
#وهيك بغش حالو  وبجيب اكيورسي عاليه كذابه تمام فا لازم نحل هالمشكله
#طيب لازم نعمل سامبل سايز لانو الامور هيك حتصير كثير كبيره بدونو ومع السموت هاض الداتا بتتضاعف ومابتحمل الرام
#ولازن مانتسخدم السامبلز سايز عادي عشان مانخرب توزيع الداتا افضل يعني
#لو استحدمت سامبلنق عادي رح نفقد كثير داتا يعني مثلا ممكن ليوز معين نفقد الاوردر تاعو رقم 3 او اورد رقم 5
#بينما باليوزر سامبلنق انا باخذ كل الداتا لاشخاص اقل بس
#  User Sampling
unique_users = final_df['user_id'].unique()

# هون السايز خليتو انتجر لانو مثيود الرانودوم تشويس لازمها عدد صحيح مش عشري
new_size = int(len(unique_users) * 0.05)

#  اخترت اليوزرز بشكل عشوائي بدون تكرار
my_users = np.random.choice(unique_users, size=new_size , replace=False)

#  فلترت الداتا خليت بس الأسطر اللي بتخص اليوزرز اللي اخترتهم
final_sample = final_df[final_df['user_id'].isin(my_users)].copy()
# هسا لازم نتاكد انو الداتا مرتبه

final_sample = final_sample.sort_values(by=["user_id","order_number"])

# هسا بعد مارتبنا مطلوب منا نعمل نقسم الداتا طيب شو في طريقه نقسمها في اكثر من طريقه لكن كلهم معقدين فا هاي ابسط اشي لحالتنا
#الي هي انو نعمل كولوم جديده نحط فيه الداتا الجديده تمام وبنقسم على ااساسها
#هون الهدف نعمل كولوم جديد فيه قيم صح وخطا الان القيم الصح او الترو بنحطها للتيست وقيم الفولس بنحطها للترين تمام ها
#الان كيف نعمل هاي القصه من خلال انو بنمرر كولم الاوردر نمبر تمام وبنقارن كل اوردر لكل يوزر مع الماكس او الطلب الاخير لهاض اليوز
#فلو كان اليوزر الطلب الي بنقارنه هو نفسه الطلب الاخير اله حيرجع ترو تمام
#هون جربت استخدم بدون ترانسفورم لاني ماكنت اعرفها طلع عندي ايرور والايرور لانو لو خليت الميثود بدونها رح ترجعلي داتا مضغوطه لانو بجيب الزبده لكل
#لانو بجيب الزبده لكل كولوم فالحل ترانسفورم عشان نطبق الماكس عكل نقطه داتا عندي تم ؟ تم

final_sample["last__orders"]= final_sample["order_number"] == final_sample.groupby("user_id")["order_number"].transform("max")
#هون قسمنا زي ما مطلوب منا
train_df = final_sample[final_sample['last__orders'] == False]
test_df = final_sample[final_sample['last__orders'] == True]
#===

# هون بدنا نقسم الداتا تاعت التريت بدنا نعطي الاكس الكولمز المهمه بس ونشيل كولوم التارقيت منها
not_for_X_columns = ['reordered', 'eval_set', 'last__orders',  'order_id',]
X_train = train_df.drop(columns=not_for_X_columns, errors='ignore')
y_train = train_df['reordered']

#نفس الشي للتيست
X_test = test_df.drop(columns=not_for_X_columns, errors='ignore')
y_test = test_df['reordered']

# Fit على التدريب فقط
train_cls = preprocessor.fit_transform(X_train , y_train)
test_cls = preprocessor.transform(X_test)
"""

target_users_count = 4000

unique_users = final_df['user_id'].unique()

np.random.seed(42)
my_users = np.random.choice(unique_users , size=target_users_count , replace=False)

final_sample = final_df[final_df['user_id'].isin(my_users)].copy()

final_sample = final_sample.sort_values(by=["user_id", "order_number"])
final_sample["last__orders"] = final_sample["order_number"] == final_sample.groupby("user_id")["order_number"].transform("max")

train_df = final_sample[final_sample['last__orders'] == False]
test_df = final_sample[final_sample['last__orders'] == True]

not_for_X_columns = ['reordered' , 'eval_set' , 'last__orders' ,  'order_id' , 'uxp_reorder_ratio', 'add_to_cart_order'   , 'order_number']

xc_train = train_df.drop(columns=not_for_X_columns , errors='ignore')
yc_train = train_df['reordered']

xc_test = test_df.drop(columns = not_for_X_columns , errors = 'ignore')
yc_test = test_df['reordered']

real_num_cols = [c for c in num_cols if c not in not_for_X_columns]
real_low_cols = [c for c in low_cols if c not in not_for_X_columns]
real_high_cols = [c for c in high_cols if c not in not_for_X_columns]

#مدامني قسمت الداتا حسب الزمن فهيك عالاغلب مش رح احتاجه
KF = KFold(n_splits = 3 , shuffle = True , random_state = 42)

preprocessor = ColumnTransformer(
    transformers=[
        ("encoding" , OneHotEncoder(handle_unknown = "ignore" , sparse_output = True , drop = "first") , real_low_cols) ,

        # Target Encoding يُستخدم مع الأعمدة الفئوية ذات عدد القيم الكبير.
        # يتم تحويل كل فئة إلى متوسط قيمة المتغير الهدف المرتبط بها.
        # لتجنب تسريب الهدف (Target Leakage)، يتم تطبيق الترميز داخل
        # الـ Cross-Validation بحيث يُحسب الترميز من بيانات التدريب فقط.
        # معاملات min_samples_leaf و smoothing تقلل تأثير الفئات النادرة
        # عبر تقريبها من المتوسط العام، مما يحد من الـ overfitting.
        ("target_encoding" , ce.TargetEncoder(min_samples_leaf = 20 , smoothing = 50) , real_high_cols) ,
        ("Frequency" , ce.CountEncoder(normalize = True) , Frequency_col) ,
        ("scaling" , StandardScaler() , real_num_cols)
                 ]
)
# معامل الفريكوانسي ترو ليش؟ , لانه اذا حطيتو فولز اللي رح يصير انه رح يوخذ عدد التكرارات زي ما هو في هيك بصير عندي تباين كبير ورح يصير بحاجه لسكيلينق
#اما هيك اللي رح يعملو انه رح يحولهم لنسبة بين ال 0 وال 1



train_cls = preprocessor.fit_transform(xc_train , yc_train)
test_cls = preprocessor.transform(xc_test)

In [ ]:
available_users = xc_train["user_id"].unique()
n_users = len(available_users)
n_users

In [ ]:
#اصريت اتأكد انه الترانسفورم تطبق
print(train_cls.shape)
print(xc_train.shape)
print()
preprocessor.get_feature_names_out()

In [ ]:
print(yc_train.value_counts(normalize=True))

plt.figure(figsize=(6 , 4))
sns.countplot(x=yc_train)
plt.title("Class Distribution in Training Set")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

#هيك بين عندي انه في تفاوت بنسب الكلاسين اللي عندي , الفرق ما بأثر كثير بس لازم اعمل موازنه لانه الدكتور طلب

In [ ]:
smote = SMOTE(random_state=42)
smote_xtrain , smote_ytrain = smote.fit_resample(train_cls, yc_train)      #بتعمل KNN
#اللي بصير هون انه السموت رح تزيد قيم الكلاس القليل بقيم ثناعيه او وهميه عشان التوازن
print(smote_ytrain.value_counts())

print("----------------------------------------------------------------")

'''rus = RandomUnderSampler(random_state = 42)
under_xtrain , under_ytrain = rus.fit_resample(train_cls , yc_train)
#هاي بتحذف من الكلاس الكبير بطريقه يصير قريب للكلاس الاقل , فيها مشكله انه ممكن تخسر بيانات مهمه من وراها واصلا هيك هيك الدقه فيها مش احسن اشي ف ما رح نستخدمها
print(under_ytrain.value_counts())'''

In [ ]:
rus = RandomUnderSampler(random_state = 42)
under_xtrain , under_ytrain = rus.fit_resample(train_cls , yc_train)
#هاي بتحذف من الكلاس الكبير بطريقه يصير قريب للكلاس الاقل , فيها مشكله انه ممكن تخسر بيانات مهمه من وراها واصلا هيك هيك الدقه فيها مش احسن اشي ف ما رح نستخدمها
print(under_ytrain.value_counts())

In [ ]:
'''model_original = RandomForestClassifier(n_estimators = 50 , max_depth = 20 , class_weight = "balanced")
#class_weight = 'balanced' هاي بتعطي اهميه اكبر للكلاس الاقل , يعني بتعاقب الموديل لما يهمله ف مجازيا بزيد وزنه , تعتبر بديل لكلشي عملناه تحت
#بدونها رح يكون في تحييز للكلاس اللي حجمه اكبر , وليش ؟ لانه الموديل بفهم انه هاض الكلاس لانه اكبر معناها هاض مهم والثاني لا
model_original.fit(train_cls , yc_train)

pred_O = model_original.predict(test_cls)
print(classification_report(yc_test , pred_O))
print(model_original.score(test_cls , yc_test))

print("----------------------------------------------------------------")

model_smote = RandomForestClassifier(n_estimators = 50 , max_depth = 20)
model_smote.fit(smote_xtrain , smote_ytrain)

pred_S = model_smote.predict(test_cls)
print(classification_report(yc_test , pred_S))
print(model_smote.score(test_cls , yc_test))

#السكور زاد بنسبه خفيفه , استدعاء الفئه اللي كانت اقل تحسن , الدقه قلت بنسبة خفيفه والسبب انه بطل يعتمد على كلاس واحد صار يعتمد على ثنين بالتصنيف ف نسبة الدقه اقل هون


print("----------------------------------------------------------------")

model_under = RandomForestClassifier(n_estimators = 50 , max_depth = 20 , random_state = 42)
model_under.fit(under_xtrain , under_ytrain)

pred_U = model_under.predict(test_cls)
print(classification_report(yc_test , pred_U))
print(model_under.score(test_cls , yc_test))'''

In [ ]:
'''fig, axes = plt.subplots(1, 3, figsize=(20, 5))

models_list = [
    (model_original, "Original (Class Weights)"),
    (model_smote, "SMOTE"),
    (model_under, "UnderSampling")
]

for i, (model, title) in enumerate(models_list):
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        test_cls,
        yc_test,
        display_labels=['Not Reordered', 'Reordered'],
        cmap=plt.cm.Blues,
        normalize='true',
        ax=axes[i]
    )
    axes[i].set_title(title)

plt.tight_layout()
plt.show()
'''
#اول مره جربت اشغل هذول عالبيانات قبل التوزيع حسب الزمن , كان تقسيم عشوائي , وطلع معي انع الالفضل سموت , بس بعد التقسيم حسب الزمن رح نعتمد الداتا الاصليه
## رجعت جربت بيانات اكبر وطلع معي السموت اعلى بنسبه بسيييييييطه وعشان هيك مش رح اهتم بالنسبه هاي ورح اعتمد الداتا الاصليه لانها ما بتوخذ وقت عكس السموت

رح نعتمد هذول بالتدريب train_cls , yc_train

وهذول بالاختبار test_cls , yc_test

Task A

In [ ]:
#هون رح اعتمد الاندر , لانه الموديل هون بصنف حسب الاقرب وبحالتي هاي ما بزبط يكون الداتا فيها عدم توازن , وما بقدر استخدم
# class_weight = 'balanced'

'''np.random.seed(42)

small_users = np.random.choice(xc_train['user_id'].unique(), size=3000, replace=False)

xtrain_tune = xc_train[xc_train['user_id'].isin(small_users)].copy()
ytrain_tune = yc_train.loc[xtrain_tune.index].copy()

xtest_tune = xc_test[xc_test['user_id'].isin(small_users)].copy()
ytest_tune = yc_test.loc[xtest_tune.index].copy()'''



np.random.seed(42)
tuning_x = xc_train.copy()
tuning_y = yc_train.copy()

t = TimeSeriesSplit(n_splits = 3)

'''trainK = []
testK = []
k_values = range(1 , 22 , 2)

for n in k_values:
    tempK = KNeighborsClassifier(n_neighbors = n , n_jobs = -1)
    Knn_model = tempK.fit(under_xtrain , under_ytrain)
    trainK.append(tempK.score(x_trainS , y_trainS))
    testK.append(tempK.score(x_testS , y_testS))

print(testK)

plt.plot(k_values , trainK , label = "Traink")
plt.plot(k_values , testK , label = "Testk")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()'''

In [ ]:
#هاض ببينلك كيف بتتغير الدقه مع زيادة \ تغيير حجم الداتا 

def learning_curves_cls(model , x_train , y_train , cv):

    train_sizes=np.linspace(0.1, 1.0, 5)

    train_sizes , train_scores , test_scores = learning_curve(
        model , 
        x_train , y_train ,
        cv = cv , n_jobs = -1 
    )

    train_mean = np.mean(train_scores , axis = 1)
    train_std  = np.std(train_scores , axis = 1)
    test_mean  = np.mean(test_scores , axis = 1)
    test_std   = np.std(test_scores , axis = 1)

    plt.figure()
    plt.title("Learning Curve (Full Data)")
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.grid()

    #  المعادله هاي عشان ارسم التباين
    plt.fill_between(train_sizes , train_mean - train_std , train_mean + train_std , alpha = 0.1 , color = "r")
    plt.fill_between(train_sizes , test_mean - test_std , test_mean + test_std , alpha = 0.1 , color = "g")

    plt.plot(train_sizes , train_mean , 'o-' , color = "r" , label = "Training score")
    plt.plot(train_sizes , test_mean , 'o-' , color = "g" , label = "Cross-validation score")

    plt.legend(loc = "best")
    plt.show()

In [ ]:
def val_curves(model , x_train , y_train , param_name , cv , param_range):

    train_scores, test_scores = validation_curve(
        model , 
        x_train , 
        y_train , 
        param_name = param_name , 
        param_range = param_range ,
        cv = cv , 
        scoring = "accuracy" ,
        n_jobs = -1
    )

    train_mean = np.mean(train_scores , axis=1)
    train_std = np.std(train_scores , axis=1)
    test_mean = np.mean(test_scores , axis=1)
    test_std = np.std(test_scores , axis=1)

    plt.figure(figsize = (10, 6))
    plt.title("Validation Curve")
    plt.xlabel("Param (Complexity)")
    plt.ylabel("score (Higher is Better)")

    plt.plot(param_range, train_mean , label = "Training Score" , color = "darkorange", lw = 2 , marker = 'o')
    plt.fill_between(param_range, train_mean - train_std , train_mean + train_std , alpha = 0.1 , color = "darkorange")

    # رسم منطقة التيست
    plt.plot(param_range , test_mean , label = "Validation Score" , color = "navy" , lw = 2, marker = 'o')
    plt.fill_between(param_range , test_mean - test_std , test_mean + test_std , alpha = 0.1 , color = "navy")

    plt.legend(loc = "best")
    plt.grid()
    plt.show()

1- KNN

In [ ]:
'''FAST_TUNING_USERS = 1000

np.random.seed(42)
# بنختار عينة جديدة صغيرة من العينة الأصلية الكبيرة
KNNsmall_users = np.random.choice(xtrain_tune['user_id'].unique(), size=800, replace=False)

# تصفية الداتا
knn_x_small = xtrain_tune[xtrain_tune['user_id'].isin(small_users)].copy()
knn_y_small = ytrain_tune[xtrain_tune['user_id'].isin(small_users)]

KNN_pipline = make_pipeline(
    preprocessor ,
    KNeighborsClassifier()
)


t = TimeSeriesSplit(n_splits = 3)

param_grid = {
    "kneighborsclassifier__n_neighbors": [7 , 10 , 20 , 25 , 30] ,
    "kneighborsclassifier__weights" : ['distance' , 'uniform']
}

KNN_grid_search = GridSearchCV(
    estimator = KNN_pipline ,
    param_grid = param_grid ,
    cv = t ,
    scoring = 'f1' ,
    n_jobs = -1 ,
    verbose=3
)

print("Done.....")

KNN_grid_search.fit(knn_x_small , knn_y_small)

print("Best CV score:" , KNN_grid_search.best_score_)
print("Best params:" , KNN_grid_search.best_params_)'''


KNNsmall_users = np.random.choice(tuning_x['user_id'].unique() , size = 2000 , replace = False)

knn_x_small = tuning_x[tuning_x['user_id'].isin(KNNsmall_users)].copy()
knn_y_small = tuning_y[tuning_x['user_id'].isin(KNNsmall_users)]

KNN_pipline = make_pipeline(
    preprocessor ,
    KNeighborsClassifier(n_jobs = -1)
)

KNNparam_grid = {
    "kneighborsclassifier__n_neighbors": [7 , 10 , 20 , 25 , 30] ,
    "kneighborsclassifier__weights" : ['distance' , 'uniform']
}

KNN_grid_search = GridSearchCV(
    estimator = KNN_pipline ,
    param_grid = KNNparam_grid ,
    cv = t ,
    scoring = "f1" ,
    n_jobs = -1
)

KNN_grid_search.fit(knn_x_small , knn_y_small)

print("\nBest Score:", KNN_grid_search.best_score_)
print("Best Parameters:", KNN_grid_search.best_params_)


In [ ]:
best_w = KNN_grid_search.best_params_['kneighborsclassifier__weights']
best_k = KNN_grid_search.best_params_['kneighborsclassifier__n_neighbors']

finalKNN_pip = make_pipeline(
        preprocessor ,
        KNeighborsClassifier(n_neighbors = best_k , weights = best_w , n_jobs = -1)
)

finalKNN_pip.fit(xc_train, yc_train)


y_pred_KNN = finalKNN_pip.predict(xc_test)

print(classification_report(yc_test , y_pred_KNN))
print(accuracy_score(yc_test , y_pred_KNN))



In [ ]:
#بعد تطبيق تقسيم زمني واقعي ومعالجة عدم التوازن، انخفضت الدقة من قيم مرتفعة غير واقعية إلى ~0.7،
#إلا أن قدرة النموذج على اكتشاف المنتجات المعاد طلبها (Recall) تحسّنت بشكل واضح،
#مما يجعل النموذج أكثر ملاءمة للاستخدام الحقيقي.

2-Logistic Regression

In [ ]:
tuning_users = np.random.choice(tuning_x["user_id"].unique() , size = 2500 , replace = False)

# تجهيز عينة للتونينق
LoR_x_small = tuning_x[tuning_x['user_id'].isin(tuning_users)].copy().sort_values(by=['user_id'])
LoR_y_small = tuning_y.loc[LoR_x_small.index]

LoR_pipline = make_pipeline(
    preprocessor ,
    LogisticRegression(solver = 'liblinear' , class_weight = 'balanced' , random_state = 42 , n_jobs = -1)
)

LoRparam_grid = {
    "logisticregression__C": [0.001 , 0.01 , 0.1 , 1 , 10] ,
    "logisticregression__penalty": ['l1' , 'l2']
}

LoR_grid_search = GridSearchCV(
    estimator = LoR_pipline ,
    param_grid = LoRparam_grid ,
    cv = t ,
    scoring = "f1" ,
    n_jobs = -1
)

LoR_grid_search.fit(LoR_x_small , LoR_y_small)

print("\nBest Score:", LoR_grid_search.best_score_)
print("Best Parameters:", LoR_grid_search.best_params_)

"""Best Score: 0.7038598274542919
Best Parameters: {'logisticregression__C': 0.001, 'logisticregression__penalty': 'l2'}"""



In [ ]:
SAMPLE_N = 75000 

xc_train_small = xc_train.sample(n=SAMPLE_N, random_state=42)
yc_train_small = yc_train.loc[xc_train_small.index]

best_C = LoR_grid_search.best_params_["logisticregression__C"]
best_P = LoR_grid_search.best_params_["logisticregression__penalty"]

final_LoR_pipeline = make_pipeline(
    preprocessor ,
    LogisticRegression(solver = 'liblinear' , C = best_C , penalty = best_P , class_weight = 'balanced' , random_state = 42 , n_jobs = -1)
)

final_LoR_pipeline.fit(xc_train_small , yc_train_small)

y_pred_LoR = final_LoR_pipeline.predict(xc_test)

print(classification_report(yc_test , y_pred_LoR))
print(f"Logistic Accuracy: {accuracy_score(yc_test , y_pred_LoR)}")

learning_curves_cls(final_LoR_pipeline , xc_train_small , yc_train_small , t)

In [ ]:
#Logistic Accuracy: 0.7295685052244335  الدقه هيك قبل ما اشوف التباين وقب ما اعدل
#Logistic Accuracy: 0.7314973303017143  هيك بعد , مش هالفرق بس كويس


In [ ]:
param_range = [0.001 , 0.01 , 0.1 , 1 , 10]
val_curves(final_LoR_pipeline , xc_train , yc_train , "logisticregression__C" , t , param_range)

In [ ]:
'''numeric_df = train_df.select_dtypes(include=['number'])
correlations = numeric_df.corrwith(train_df['reordered']).sort_values(ascending=False)

print(correlations.head(10))'''

In [ ]:
lr_model = final_LoR_pipeline.named_steps['logisticregression'] 
preprocessor = final_LoR_pipeline.named_steps['columntransformer'] 

try:
    feature_names = preprocessor.get_feature_names_out()
    feature_names = [name.split('__')[-1] for name in feature_names]
except:
    feature_names = [f"Feature_{i}" for i in range(len(lr_model.coef_[0]))]

if len(feature_names) != len(lr_model.coef_[0]):
    print(f"Mismatch! Names: {len(feature_names)}, Coefs: {len(lr_model.coef_[0])}")
    feature_names = [f"Feat_{i}" for i in range(len(lr_model.coef_[0]))]

coefs = pd.DataFrame({
    'Feature': feature_names ,
    'Coefficient': lr_model.coef_[0]
})

coefs['Abs_Value'] = coefs['Coefficient'].abs()
top_20_coefs = coefs.sort_values(by = 'Abs_Value' , ascending = False).head(20)

plt.figure(figsize = (10 , 8))
colors = ['blue' if x > 0 else 'red' for x in top_20_coefs['Coefficient']]
plt.barh(top_20_coefs['Feature'] , top_20_coefs['Coefficient'] , color = colors)

plt.title("Top 20 Coefficients in Logistic Regression (L2 Regularization)")
plt.xlabel("Coefficient Value (Impact on Log-Odds)")
plt.ylabel("Feature")
plt.gca().invert_yaxis()
plt.grid(axis='x' , linestyle = '--' , alpha = 0.7)
plt.show()

In [ ]:
X_train_sample = xc_train.sample(n = 100 , random_state = 42)
X_train_transformed = preprocessor.transform(X_train_sample)

X_test_sample_lin = xc_test.sample(n = 1000 , random_state = 42)
X_test_transformed_lin = preprocessor.transform(X_test_sample_lin)

explainer_linear = shap.LinearExplainer(lr_model , X_train_transformed)
shap_values_linear = explainer_linear.shap_values(X_test_transformed_lin)

plt.figure()
shap.summary_plot(shap_values_linear , X_test_transformed_lin , feature_names = feature_names , show = False)
plt.title("Logistic Regression SHAP Summary" , fontsize = 16)
plt.show()

3- Random Forest

In [ ]:
tuning_users = np.random.choice(tuning_x["user_id"].unique(), size = 800 , replace = False)
RF_x_small = tuning_x[tuning_x['user_id'].isin(tuning_users)].copy().sort_values(by = ['user_id'])
RF_y_small = tuning_y.loc[RF_x_small.index]
 
RF_pipline = make_pipeline(
    preprocessor ,
    RandomForestClassifier(class_weight = "balanced" , random_state = 42 , n_jobs = -1)
)

RFparam_grid = {
    "randomforestclassifier__n_estimators": [100 , 150 , 200] ,
    "randomforestclassifier__max_depth": [None , 10 , 20] ,
    "randomforestclassifier__min_samples_split": [2 , 5 , 10] ,
    "randomforestclassifier__min_samples_leaf": [1 , 2 , 4] ,
    "randomforestclassifier__class_weight": ['balanced' , 'balanced_subsample']
}

RF_grid_search = GridSearchCV(
    estimator = RF_pipline ,
    param_grid = RFparam_grid ,
    cv = t ,
    scoring = "f1" ,
    n_jobs = -1
)

RF_grid_search.fit(RF_x_small , RF_y_small)

print("\nBest Score:", RF_grid_search.best_score_)
print("Best Parameters:", RF_grid_search.best_params_)

In [ ]:
best_N = RF_grid_search.best_params_["randomforestclassifier__n_estimators"]
best_MD = RF_grid_search.best_params_["randomforestclassifier__max_depth"]
best_MSP = RF_grid_search.best_params_[ "randomforestclassifier__min_samples_split"]
best_MSL = RF_grid_search.best_params_["randomforestclassifier__min_samples_leaf"]
best_W = RF_grid_search.best_params_["randomforestclassifier__class_weight"]

final_RF_pipeline = make_pipeline(
    preprocessor,
    RandomForestClassifier(n_estimators = best_N , max_depth = best_MD ,
                    min_samples_leaf = best_MSL , min_samples_split = best_MSP ,
                 class_weight = best_W , n_jobs = -1 , random_state = 42
                        )
)



final_RF_pipeline.fit(xc_train , yc_train)

y_pred_RF = final_RF_pipeline.predict(xc_test)

print(classification_report(yc_test , y_pred_RF))
print(f"Random Forest Accuracy: {accuracy_score(yc_test , y_pred_RF)}")

learning_curves_cls(final_RF_pipeline , xc_train , yc_train , t)

In [ ]:
param_range = [1, 3, 5, 8, 10, 15, 20]
val_curves(final_RF_pipeline , xc_train , yc_train , "randomforestclassifier__max_depth" , t , param_range)

4- Gradient boosting classifier

In [ ]:
# رح نسخدم هالمره الترين سي ل س عشان معمولها بري بروسس وكلها ارقام لانو لو استخدمنا الداتا العاديه رح نتطر نحذ الكولومز الي مش رقميه وبالتالي نخسر بيانات مهمه
tuning_users_ids = np.random.choice(xc_train['user_id'].unique(), size=1000, replace=False)


# بما أن train_cls يطابق xc_train سطراً بسطر، نستخدم شرط xc_train لقص train_cls
XGB_x_small = train_cls[xc_train['user_id'].isin(tuning_users_ids)]
XGB_y_small = yc_train[xc_train['user_id'].isin(tuning_users_ids)]

ratio = float(np.sum(XGB_y_small == 0)) / np.sum(XGB_y_small == 1)

print(f"Tuning shape: {XGB_x_small.shape}")
print(f"Calculated Ratio: {ratio:.2f}")

In [ ]:
 # تعريف الموديل مباشرة بدون Pipeline لأن الداتا جاهزة
xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    scale_pos_weight=ratio,
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1
)

XGBparam_grid = {
    "learning_rate": [0.01, 0.1, 0.2],
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 5, 7]
}


XGB_grid_search = GridSearchCV(
    estimator = xgb_model,
    param_grid = XGBparam_grid,
    cv = 3,
    scoring = "f1",
    n_jobs = -1,

)


XGB_grid_search.fit(XGB_x_small, XGB_y_small)

print("\nBest Score (F1):", XGB_grid_search.best_score_)
print("Best Parameters:", XGB_grid_search.best_params_)
# بعد التعديل ارتفعت السكور من 72 ل 77
# Best Score (F1): 0.7726193292101673
# Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}

In [ ]:
best_LR = XGB_grid_search.best_params_["learning_rate"]
best_NE = XGB_grid_search.best_params_["n_estimators"]
best_MD = XGB_grid_search.best_params_["max_depth"]

# print(f"Learning Rate: {best_LR}, N_Estimators: {best_NE}, Max_Depth: {best_MD}")

In [ ]:
full_ratio = float(np.sum(yc_train == 0)) / np.sum(yc_train == 1)
print(f"Full Data Ratio: {full_ratio:.2f}")


#  #  الموديل النهائي
# final_xgb = XGBClassifier(
#     learning_rate = best_LR,
#     n_estimators = best_NE,
#     max_depth = best_MD,
#     scale_pos_weight = full_ratio,
#     objective = 'binary:logistic',
#     eval_metric = 'logloss',
#     use_label_encoder = False,
#     random_state = 42,
#     n_jobs=-1
# )

# # على كل الداتا
# final_xgb.fit(train_cls, yc_train)


#========================================================
# عشان نعمل خظوه الايرلي سبلتنق لازم نقسم الداتا كمان مره ولازم ننتبه يكون تقسيمنا براعي الزمن الي قسمناه بالبدايه تمام
#لانو الايرلي ستوبنق بحتاج نعمل فاليديشن اكثر من مره حتى يعرف متى يوقف
# بنجيب كل اليوزرز من الداتا الأصلية وبنقسمهم 90% تدريب 10% مراقبة
train_users, val_users = train_test_split(xc_train['user_id'].unique(), test_size=0.10, random_state=42)

# 2. عمل الماسك (Masks) لتحديد أماكن هدول اليوزرز في الداتا
# train_mask: بيحدد أسطر يوزرز التدريب
# val_mask: بيحدد أسطر يوزرز المراقبة
train_mask = xc_train['user_id'].isin(train_users).values
val_mask = xc_train['user_id'].isin(val_users).values

# 3. تطبيق الماسك على الداتا المعالجة الجاهزة (train_cls)
# هيك ضمنا إن X_tr و X_val مفصولين تماماً بناءً على الأشخاص
X_tr = train_cls[train_mask]
y_tr = yc_train[train_mask]

X_val = train_cls[val_mask]
y_val = yc_train[val_mask]

print(f"Train shapes: {X_tr.shape}, Val shapes: {X_val.shape}")


final_xgb = xgb.XGBClassifier(
    learning_rate = best_LR,
    n_estimators = 3000,         # رقم كبير جدا نعطيه راحته
    max_depth = best_MD,
    # scale_pos_weight = full_ratio,
    objective = 'binary:logistic',
    eval_metric = 'logloss',
    use_label_encoder = False,
    early_stopping_rounds = 50,  #وقف اذا ماتحسن بعد 50 راوند
    random_state = 42,
    n_jobs = -1
)

final_xgb.fit(
    X_tr, y_tr,
    eval_set=[(X_tr, y_tr), (X_val, y_val)],
    verbose=False
)

print("-" * 30)
print(f"Training Stopped at iteration: {final_xgb.best_iteration}")
print(f"Best Validation Score: {final_xgb.best_score}")

In [ ]:
y_pred_xgb = final_xgb.predict(test_cls)

print(" XGBoost Report")
print(classification_report(yc_test, y_pred_xgb))

print(f"XGBoost Accuracy: {final_xgb.score(test_cls, yc_test)}")
# هاي النتائج قبل الايرلي ستوبنق
# XGBoost Report
#               precision    recall  f1-score   support

#            0       0.50      0.71      0.59     23041
#            1       0.85      0.70      0.77     55245

#     accuracy                           0.70     78286
#    macro avg       0.67      0.71      0.68     78286
# weighted avg       0.75      0.70      0.71     78286

# XGBoost Accuracy: 0.702922617070740

# النتائج هاي بعد الايرلي ستوبنق وفرقت معنا منيح
# XGBoost Report
#               precision    recall  f1-score   support

#            0       0.51      0.74      0.60     23041
#            1       0.87      0.70      0.77     55245

#     accuracy                           0.71     78286
#    macro avg       0.69      0.72      0.69     78286
# weighted avg       0.76      0.71      0.72     78286

# XGBoost Accuracy: 0.7107784278159569
#بنلاحظ الاكيروسي زادت تقريبا 1 بالميه
# وال ري كول لكلاس 0 تحسن بشكل منيح من 71 ل 74 يعني المودل صار احسن بالي مابشترو
#======================================


# بعد اضافه تو كولمز مهمين

#  XGBoost Report
#               precision    recall  f1-score   support

#            0       0.75      0.97      0.85     23041
#            1       0.98      0.87      0.92     55245

#     accuracy                           0.90     78286
#    macro avg       0.87      0.92      0.88     78286
# weighted avg       0.92      0.90      0.90     78286

In [ ]:

print("Plotting Feature Importance")

try:

    feature_names = preprocessor.get_feature_names_out()

    clean_names = [name.split('__')[-1] for name in feature_names]

    final_xgb.get_booster().feature_names = clean_names

    fig, ax = plt.subplots(figsize=(12, 10))
    xgb.plot_importance(
        final_xgb,
        ax=ax,
        max_num_features=20,     
        height=0.5,
        importance_type='weight', 
        color='teal',
        title="Top 20 Features Influencing Reorders (XGBoost)"
    )
    plt.show()

except Exception as e:
    print(f"Could not extract feature names automatically (Error: {e}). Plotting by index instead...")

    fig, ax = plt.subplots(figsize=(10, 8))
    xgb.plot_importance(final_xgb, ax=ax, max_num_features=20, height=0.5, color='teal')
    plt.title("Top 20 Features (by Index)")
    plt.show()

In [ ]:
xgboost_model = final_xgb 

X_test_sample = xc_test.sample(n=1000, random_state=42)
y_test_sample = yc_test.loc[X_test_sample.index]

X_test_transformed = preprocessor.transform(X_test_sample)

try:
    feature_names = preprocessor.get_feature_names_out()
    feature_names = [name.split('__')[-1] for name in feature_names]
except:
    print("Could not extract feature names automatically.")
    feature_names = [f"Feature_{i}" for i in range(X_test_transformed.shape[1])]


print("Calculating SHAP values")

explainer = shap.TreeExplainer(xgboost_model)
shap_values = explainer.shap_values(X_test_transformed)


print("Plotting SHAP Summary Plot")
plt.figure()
shap.summary_plot(shap_values, X_test_transformed, feature_names=feature_names, show=False)
plt.title("XGBoost Feature Importance (SHAP)", fontsize=16)
plt.show()


mean_shap = np.abs(shap_values).mean(axis=0)
top_3_indices = np.argsort(mean_shap)[-3:] 

print("Plotting Dependence Plots...")
for idx in top_3_indices:
    shap.dependence_plot(idx, shap_values, X_test_transformed, feature_names=feature_names)


print("Generating Local Explanation (Force Plot)...")

preds = xgboost_model.predict(X_test_transformed)
actual = y_test_sample.values

try:
    target_idx = np.where((preds == 1) & (actual == 1))[0][0]
    
    print(f"Explanation for sample index {target_idx} (Correct Prediction)")
    
    shap.force_plot(
        explainer.expected_value, 
        shap_values[target_idx], 
        X_test_transformed[target_idx], 
        feature_names=feature_names,
        matplotlib=True
    )
except IndexError:
    print("لم يتم العثور على حالة True Positive في العينة المختارة.")



    #الرسمه الاولى بتبينلك قديش تأثير العامود بقرار التنبؤ , يعني اذا كان موجب رايح عاليمين بكون العامود هاض كثير اثر على اعادة الشراء والعكس صحيح
    #كل ما زادت عدد طلبات الزبون بزيد احتمالية اعادة الشراء

    #الرسمه الرابعه , بتوضح انه واحد من العوامل المهمه كثير لاعادة الشراء انه يكون الزبون مشتري نفس المنتج من قبل

    #الرسمه الثالثه بتوضح انه في علاقة عكسية , بمعنى انه مثلا اذا كان الزبون مشتري مبارح ف صعب انه يرجع يشتري كمان مره بعد فتره قصيره نسبيا

    #الصوره الثانيه بتحكي انه مثلا الزبائن الجداد بكون هدفهم يشترو شغلات اساسيه ف عشان هيك بكون في تكرار كثير بالشراء 
    #اما الزبائن القدام بكونو بشترو شغلات جديده عن العاده عشان هيك نسبة اعادة الشراء قليله نسبيا

    #الرسمه الاخيره , مسكنا زبون واحد وشفنا ليش الموديل حكى انه بنسبة 85 بالميه رح يكرر الشراء
    #الاعتماد كان على :

    """
    days_since_prior_order: الزبون صرله فترة مش طالب (القيمة عالية)، فهاد دفش الاحتمالية لفوق.

    uxp_total_bought: الزبون شاري هذا المنتج كثير مرات قبل هيك.

    85 هذا الزبون بيحب هذا المنتج (شراه كثير)، وصرله فترة مش طالبه (خلص من عنده)، لذلك الموديل توقع الشراء بنسبة                  
    """


In [ ]:
models = {
    "Logistic Regression": final_LoR_pipeline ,
    "Random Forest": final_RF_pipeline ,   
    "KNN Classifier": finalKNN_pip  
}

results = []

for name , model in models.items():
    print(f"Testing {name}")
    scores = cross_val_score(model , xc_train , yc_train , cv = 5 , scoring = 'accuracy' , n_jobs = -1)
    
    for score in scores:
        results.append({"Model": name , "Accuracy": score})

df_results = pd.DataFrame(results)

plt.figure(figsize = (12 , 6))
sns.set_style("whitegrid")
sns.boxplot(x = "Model" , y = "Accuracy" , data = df_results , palette = "viridis" , width = 0.5)
sns.stripplot(x = "Model" , y = "Accuracy" , data = df_results , color = 'black' , alpha = 0.5 , jitter = True)
plt.title('Classification Models Benchmark (5-Fold CV)' , fontsize = 15)
plt.ylabel('Accuracy Score' , fontsize = 12)
plt.ylim(0.6 , 1.0)  
plt.show()

#هالرسمه العاطله بتبينلك مقارنه بدقة كل موديل 
#اللوجيستيك الخط تبعه كثير رفيع ف هاض بدل على انه مستقر وشو ما عملت فيه ما رح يزيد
#الراندوم وضعها نار شرار واتوقع لو اني حطيت ال سي في 10 كان رح يعطيني احتمالية اكيوريسي اعلى بس جهازي رح يفقع طبعا
#ال كنن زباله ورح يضل طول عمره زباله

RF: 0.7339 | LoR: 0.7066 | KNN: 0.7227  قبل

RF: 0.8551 | LoR: 0.7295 | KNN: 0.7630  بعد

Task B

In [ ]:
#اول مره عملنا سبلت عملناه على اساس انه التارقيت كل كم اليوزر بشتري , وكانت النتائج كثير سيئه , ممكن الشغل نفسه كان غلط ابصر
#بس بعديه حذفناه كامل ورجعنا قسمنا هيك وحلينا كل المشاكل

cleaned_df = final_df[final_df['order_number'] > 1].copy()
orders_level = (
    cleaned_df
    .sort_values(["user_id" , "order_number"])      
    .drop_duplicates("order_id" , keep="last")      
    .copy()
)


target_users_count = 30000
unique_users = orders_level["user_id"].unique()

if target_users_count > len(unique_users):
    target_users_count = len(unique_users)

np.random.seed(42)
my_users = np.random.choice(unique_users, size=target_users_count, replace=False)

final_sample = orders_level[orders_level["user_id"].isin(my_users)].copy()
final_sample = final_sample.sort_values(by=["user_id", "order_number"])


final_sample["last__orders"] = (
    final_sample["order_number"]
    == final_sample.groupby("user_id")["order_number"].transform("max")
)

train_df = final_sample[final_sample["last__orders"] == False].copy()
test_df  = final_sample[final_sample["last__orders"] == True].copy()

y_col = "days_since_prior_order"
#احتياط المفروض
train_df = train_df.dropna(subset=[y_col])
test_df = test_df.dropna(subset=[y_col])

not_for_X_columns = [
    "eval_set", "last__orders", "order_id",
    "reordered", "add_to_cart_order" , y_col
]
#order_number
xr_train = train_df.drop(columns=not_for_X_columns + ["user_id"], errors="ignore")
yr_train = train_df[y_col].astype("float32")

xr_test  = test_df.drop(columns=not_for_X_columns + ["user_id"], errors="ignore")
yr_test  = test_df[y_col].astype("float32")



real_num_col_reg = xr_train.select_dtypes(include=["int32", "float32", "float64"]).columns.tolist()

real_num_col_reg = [c for c in real_num_col_reg if c not in id_cols]

real_low_col_reg  = [c for c in low_cols  if c in xr_train.columns]
real_high_col_reg = [c for c in high_cols if c in xr_train.columns]
real_freq_col_reg = [c for c in Frequency_col if c in xr_train.columns]



preprocessor_reg = ColumnTransformer(
    transformers=[
        ("encoding", OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first"), real_low_col_reg),
        ("target_encoding", ce.TargetEncoder(min_samples_leaf=20, smoothing=50), real_high_col_reg),
        ("frequency", ce.CountEncoder(normalize=True), real_freq_col_reg),
        ("scaling", StandardScaler(), real_num_col_reg),
    ],
    remainder="drop"
)

train_reg = preprocessor_reg.fit_transform(xr_train, yr_train)
test_reg  = preprocessor_reg.transform(xr_test)

In [ ]:
print(f"xr_train shape : {xr_train.shape}")
print(f"train_reg shape : {train_reg.shape}")

1- KNN

In [ ]:
np.random.seed(42)
tuning_xr = xr_train.copy()
tuning_yr = yr_train.copy()
GF = GroupKFold(n_splits=5)

tuning_groups = train_df["user_id"].copy()




In [ ]:
def val_curves(model , x_train , y_train , param_name , cv , param_range):

    train_scores, test_scores = validation_curve(
        model , 
        x_train , 
        y_train , 
        param_name = param_name , 
        param_range = param_range ,
        cv = cv , 
        scoring = "neg_root_mean_squared_error" , 
        n_jobs = -1
    )

    train_mean = abs(np.mean(train_scores , axis=1))
    train_std = np.std(train_scores , axis=1)
    test_mean = abs(np.mean(test_scores , axis=1))
    test_std = np.std(test_scores , axis=1)

    plt.figure(figsize = (10, 6))
    plt.title("Validation Curve")
    plt.xlabel("param (Complexity)")
    plt.ylabel("RMSE Error (Lower is Better)")

    plt.plot(param_range, train_mean , label = "Training Error" , color = "darkorange", lw = 2 , marker = 'o')
    plt.fill_between(param_range, train_mean - train_std , train_mean + train_std , alpha = 0.1 , color = "darkorange")

    # رسم منطقة التيست
    plt.plot(param_range , test_mean , label = "Validation Error" , color = "navy" , lw = 2, marker = 'o')
    plt.fill_between(param_range , test_mean - test_std , test_mean + test_std , alpha = 0.1 , color = "navy")

    plt.legend(loc = "best")
    plt.grid()
    plt.show()

In [ ]:
def learning_curves_reg(model , x_train , y_train , cv ):

    train_sizes=np.linspace(0.1, 1.0, 5)

    train_sizes , train_scores , test_scores = learning_curve(
        model , 
        x_train , y_train ,
        cv = cv , n_jobs = -1 ,
        scoring='neg_root_mean_squared_error' ,
    )

    train_mean = abs(np.mean(train_scores , axis = 1))
    train_std  = np.std(train_scores , axis = 1)
    test_mean  = abs(np.mean(test_scores , axis = 1))
    test_std   = np.std(test_scores , axis = 1)

    plt.figure()
    plt.title("Learning Curve (Full Data)")
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.grid()

    #  المعادله هاي عشان ارسم التباين
    plt.fill_between(train_sizes , train_mean - train_std , train_mean + train_std , alpha = 0.1 , color = "r")
    plt.fill_between(train_sizes , test_mean - test_std , test_mean + test_std , alpha = 0.1 , color = "g")

    plt.plot(train_sizes , train_mean , 'o-' , color = "r" , label = "Training score")
    plt.plot(train_sizes , test_mean , 'o-' , color = "g" , label = "Cross-validation score")

    plt.legend(loc = "best")
    plt.show()

In [ ]:
def Q_Q(model , xr_train , yr_train , xr_test , yr_test):

    model.fit(xr_train , yr_train)
    y_pred = model.predict(xr_test)
    residuals = yr_test - y_pred

    plt.figure(figsize=(8, 6))
    stats.probplot(residuals, dist="norm", plot=plt)
    plt.title('Q-Q Plot for Residuals Normality')
    plt.grid(True)
    plt.show()

In [ ]:
def Residual_plots(model , xr_train , yr_train , xr_test , yr_test):

    model.fit(xr_train , yr_train)
    y_pred = model.predict(xr_test)

    #الفرق بين القيم الاصليه والمتوقعه , الهدف منها انك تشوف التباين 
    residuals = yr_test - y_pred

    plt.figure(figsize = (10 , 5))
    sns.scatterplot(x = y_pred , y = residuals)
    plt.axhline(y = 0 , color = 'r' , linestyle = '--')   
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title('Residual Plot')
    plt.show()

In [ ]:
def model_score(model , xr_train , yr_train , xr_test , yr_test):
    model.fit(xr_train , yr_train)
    y_pred = model.predict(xr_test)

    mae = mean_absolute_error(yr_test , y_pred)
    print(f"MAE:  {mae}")

    rmse = np.sqrt(mean_squared_error(yr_test , y_pred))
    print(f"RMSE: {rmse}")

    r2 = r2_score(yr_test , y_pred)
    print(f"R2:   {r2}")

    mse = mean_squared_error(yr_test, y_pred)   
    print(f"MSE:  {mse}")         

    #اللي بتعمله هاي انها بتعمل خصم على كل فيتشر جديد بدخل للموديل , اذا الفيتشر الجديد زاد السكور معناها انه فيه معلومات مهمه ومفيده للموديل , واذا العكس معناها انه زباله وحكي فاضي
    n = len(yr_test)
    p = xr_test.shape[1]   
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    print(f"Adj R2: {adj_r2}")

In [ ]:
def het_breu(model , xr_train , yr_train , test_reg , yr_test):

    model.fit(xr_train , yr_train)
    y_pred = model.predict(xr_test)
    residuals = yr_test - y_pred
    xtest_with_const = sm.add_constant(test_reg)

    lm , p_value , fvalue , f_pvalue = het_breuschpagan(residuals , xtest_with_const)

    """
    lm-> Lagrange Multiplier Statistic   بقيس قوة العلاقه بين مربع الاخطاء والمتغيرات المستقله
    p_value-> القيمه الاختماله الخاصه ب lm 
    fvalue-> F-statistic
    f_pvalue-> نفس الاولى
    #رح استخدم الاولى لانها ادق على البانات الكبيره من الاخيره
    """

    if p_value < 0.05 :
        print("Heteroscedasticity") #تباين سيء ومش ثابت

    else:
        print("Homoscedasticity")   #تباين ثابت

In [ ]:
KNNRsmall_users = np.random.choice(tuning_groups.unique(), size=5000, replace=False)

mask = tuning_groups.isin(KNNRsmall_users)

KNNR_x_small = tuning_xr.loc[mask].copy()
KNNR_y_small = tuning_yr.loc[mask].copy()
KNNR_groups_small = tuning_groups.loc[mask].copy()

KNNR_pipline = make_pipeline(
    preprocessor_reg ,
    KNeighborsRegressor()
)
 #  [7 , 10 , 20 , 25 , 30] هون اول اشي عملنا اخترنا
#تمام بما انو المودل اختار 30 هاض بيعني انو لسا في مجال نضيف عدد اكبر من النيبرز فا رح نعدلها ل [30, 50, 70, 100]
# لو بدك تاكد تقلل ال 100 لو بتعمل اندر فيت وشوف
KNNRparan_grid = {
    "kneighborsregressor__n_neighbors": [30, 50, 70, 100] ,
    "kneighborsregressor__weights" : ['distance' , 'uniform']
}

KNNR_grid_search = GridSearchCV(
    estimator = KNNR_pipline ,
    param_grid = KNNRparan_grid ,
    cv = GF ,
    scoring = "r2" ,
    n_jobs = -1
)


KNNR_grid_search.fit(KNNR_x_small , KNNR_y_small , groups = KNNR_groups_small)

print("\nBest Score:" , KNNR_grid_search.best_score_)
print("Best Parameters:" , KNNR_grid_search.best_params_)
#النتائج هاي قبل تزبيط السبلت
#"""Best Score: -8.939542770385742        neg_root_mean_squared_error
#Best Parameters: {'kneighborsregressor__n_neighbors': 30, 'kneighborsregressor__weights': 'uniform'}"""
#=============================================
#النتائج هاي بعد تزبيط السبلت وعدد الجيران
#Best Score: 0.19590063095092775
#Best Parameters: {'kneighborsregressor__n_neighbors': 100, 'kneighborsregressor__weights': 'uniform'}

In [ ]:
best_Rk = KNNR_grid_search.best_params_["kneighborsregressor__n_neighbors"]

best_Rw = KNNR_grid_search.best_params_["kneighborsregressor__weights"]

final_KNNR_pip = make_pipeline(
    preprocessor_reg ,
    KNeighborsRegressor(n_neighbors = best_Rk , weights = best_Rw , n_jobs = -1)
)

model_score(final_KNNR_pip , xr_train , yr_train , xr_test , yr_test)
Residual_plots(final_KNNR_pip , xr_train , yr_train , xr_test , yr_test)

het_breu(final_KNNR_pip , xr_train , yr_train , test_reg , yr_test)

#قبل التعديل على السبلت وعدد الجيران هيك كانت النتائج
# MAE:  5.356390476226807
# RMSE: 7.753037626488352
# R2:   0.3814122676849365

#==============================
#بعد التعديل
# MAE:  4.928064823150635
# RMSE: 6.896374702453613
# R2:   0.5105602741241455
#شوف فرق ال ار 2 بعد التعديل مش بطال على كي ان ان على داتا زي هيك

In [ ]:
Q_Q(final_KNNR_pip , xr_train , yr_train , xr_test , yr_test)

2- Decision Tree Regressor

In [ ]:
DTRsmall_users = np.random.choice(tuning_groups.unique(), size=5000, replace=False)
mask = tuning_groups.isin(DTRsmall_users)

DTR_x_small = tuning_xr.loc[mask].copy()
DTR_y_small = tuning_yr.loc[mask].copy()
DTR_groups_small = tuning_groups.loc[mask].copy()


DTR_pipline = make_pipeline(
    preprocessor_reg ,
    DecisionTreeRegressor(random_state = 42)
)

DTRparam_grid = {
    "decisiontreeregressor__max_depth": [None , 10 , 20 , 30] ,
    "decisiontreeregressor__min_samples_split": [2 , 10 , 20] ,
    "decisiontreeregressor__min_samples_leaf": [1 , 10 , 50] ,
}

#ال KF اعطتني نفس نتيجة ال GF
DTR_grid_search = GridSearchCV(
    estimator = DTR_pipline ,
    param_grid = DTRparam_grid ,
    cv = KF ,
    scoring = "r2" ,
    n_jobs = -1
)

DTR_grid_search.fit(DTR_x_small, DTR_y_small, groups = DTR_groups_small)

print("\nBest Score:" , DTR_grid_search.best_score_)
print("Best Parameters:" , DTR_grid_search.best_params_)

In [ ]:
best_RMD = DTR_grid_search.best_params_["decisiontreeregressor__max_depth"]
best_RMSS = DTR_grid_search.best_params_[ "decisiontreeregressor__min_samples_split"]
best_RMSL = DTR_grid_search.best_params_["decisiontreeregressor__min_samples_leaf"]

final_DTR_pip = make_pipeline(
    preprocessor_reg ,
    DecisionTreeRegressor(max_depth = best_RMD , min_samples_leaf = best_RMSL ,
                           min_samples_split = best_RMSS , random_state = 42)
)
model_score(final_DTR_pip , xr_train , yr_train , xr_test , yr_test)
Residual_plots(final_DTR_pip , xr_train , yr_train , xr_test , yr_test)

het_breu(final_DTR_pip , xr_train , yr_train , test_reg , yr_test)

#قبل تعديل السبلت
# MAE:  5.257224113534276
# RMSE: 7.805453685792402
# R2:   0.3730198334737874
 #بعد التعديل
#MAE:  4.519589961657373
#RMSE: 6.618155207417279
#R2:   0.5464313036296291
#MSE:  43.799978349464446
#Adj R2: 0.5463093765607123

In [ ]:
Q_Q(final_DTR_pip , xr_train , yr_train , xr_test , yr_test)

In [ ]:
learning_curves_reg(final_DTR_pip , xr_train , yr_train , KF)

In [ ]:
#اخذت اهم باراميتر واكثر واحد بأثر على تعقيد الموديل
#اخذت القيم زي ما همه من التونينق بس بدونو النان , اعطاني عليها ايرور
depth_range = [1, 3, 5, 7, 10, 15, 20 , 30]

val_curves(final_DTR_pip , xr_train , yr_train , "decisiontreeregressor__max_depth" , KF , depth_range)

3- Random Forest

In [ ]:
RFRsmall_users = np.random.choice(tuning_groups.unique(), size=1000, replace=False)
mask = tuning_groups.isin(RFRsmall_users)

RFR_x_small = tuning_xr.loc[mask].copy()
RFR_y_small = tuning_yr.loc[mask].copy()
RFR_groups_small = tuning_groups.loc[mask].copy()

RFR_pipline = make_pipeline(
    preprocessor_reg ,
    RandomForestRegressor(random_state = 42 , n_jobs = -1)
)

RFRparam_grid = {
    "randomforestregressor__n_estimators": [100 , 150 , 200],
    "randomforestregressor__max_depth": [None , 10 , 20],
    "randomforestregressor__min_samples_leaf": [1, 2, 5] ,
    "randomforestregressor__min_samples_split": [2 , 5 , 10]
}

RFR_grid_search = GridSearchCV(
    estimator = RFR_pipline ,
    param_grid = RFRparam_grid ,
    cv = GF ,
    scoring = "r2" ,
    n_jobs = -1
)

RFR_grid_search.fit(RFR_x_small, RFR_y_small, groups = RFR_groups_small)

print("\nBest Score:" , RFR_grid_search.best_score_)
print("Best Parameters:" , RFR_grid_search.best_params_)

"""Best Score: 0.1056874835441742
Best Parameters: {'randomforestregressor__max_depth': 10, 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__min_samples_split': 10, 'randomforestregressor__n_estimators': 200}"""

In [ ]:
best_N = RFR_grid_search.best_params_["randomforestregressor__n_estimators"]
best_MD = RFR_grid_search.best_params_[ "randomforestregressor__max_depth"]
best_MSL = RFR_grid_search.best_params_["randomforestregressor__min_samples_leaf"]
best_MSS = RFR_grid_search.best_params_["randomforestregressor__min_samples_split"]

final_RFR_pip = make_pipeline(
    preprocessor_reg ,
    RandomForestRegressor(n_estimators = best_N , max_depth = None ,
                    min_samples_leaf = 10 , min_samples_split = best_MSS ,
                    n_jobs = -1 , random_state = 42 , criterion="squared_error"
                        )
)

#criterion -> CART algorithm قبل شوي قرأتها بالسلايدات يسعد مساك دكتور
#وللأسف الشديد ما فرقت معي بالنتيجه نهائيا

#الماكس ديبث والسامبل ليف , حطيتهم بشكل يدوي بعد تجارب كثيره , لانه الموديل صارله بالبدايه صارله اندر فيت , بعدين زدت سبة الداتا وصار
#اندر فيت , ف ضلييت اعدل واجرب لتني اعتمدت هذول 
#رفعت السامبلز من 10000 الى 30000 :)

model_score(final_RFR_pip , xr_train , yr_train , xr_test , yr_test) 
Residual_plots(final_RFR_pip , xr_train , yr_train , xr_test , yr_test)

r2_train = final_RFR_pip.score(xr_train, yr_train)

het_breu(final_RFR_pip , xr_train , yr_train , test_reg , yr_test)

""""MAE:  4.53925179592098
RMSE: 6.636387873223536
R2:   0.5439287458669073
MSE:  44.04164400386841
Adj R2: 0.5438061460674092"""

In [ ]:
Q_Q(final_RFR_pip , xr_train , yr_train , xr_test , yr_test)